In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import os

In [5]:
epoch_num = 5
sequence_length = 8  # Number of time steps to consider

# Define file and directory names
file_name = '0_61938.csv'
data_dir = 'ohlc_data'
# parent_dir = 'forexPredictor'
# repo = 'Repos_git'
# repo_dir = 'Make_Money_with_Tensorflow_2.0'
# Get the absolute base directory dynamically
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move up one level

# Construct the full file path in an OS-independent way
# data_path = os.path.join(base_dir, repo, repo_dir, parent_dir, data_dir, file_name)
data_path = os.path.join(base_dir, data_dir, file_name)

# Check if the file exists before using it
if os.path.exists(data_path):
    print(f"✅ File found: {data_path}")
else:
    print(f"❌ Error: File not found at {data_path}")


ucols=['Open', 'High', 'Low', 'Close']
data_main = pd.read_csv(data_path, usecols=ucols)
data_main.reset_index(drop=True, inplace=True)
data_main.head()


❌ Error: File not found at /home/iftekhar/Make_Money_with_Tensorflow_2.0/forexPredictor/ohlc_data/0_61938.csv


FileNotFoundError: [Errno 2] No such file or directory: '/home/iftekhar/Make_Money_with_Tensorflow_2.0/forexPredictor/ohlc_data/0_61938.csv'

In [ ]:
# 1. Load your OHLC time series dataset
df = pd.read_csv("ohlc_data.csv", parse_dates=['Date'], index_col='Date')
df = df[['Open', 'High', 'Low', 'Close']]  # Keep only OHLC

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
window_size = 20

# 2. Create sequences for time series
def create_sequences(data, window):
    sequences = []
    for i in range(len(data) - window):
        sequences.append(data[i:i + window])
    return np.array(sequences)

sequences = create_sequences(scaled_data, window_size)

# 3. Augmentation for positive/negative pairs
def augment_series(series):
    noise = np.random.normal(0, 0.01, series.shape)
    jittered = series + noise
    return jittered

def generate_ssl_triplets(sequences):
    anchors = []
    positives = []
    negatives = []
    for i in range(len(sequences)):
        anchor = sequences[i]
        positive = augment_series(anchor)
        negative_idx = (i + np.random.randint(10, 100)) % len(sequences)
        negative = sequences[negative_idx]
        anchors.append(anchor)
        positives.append(positive)
        negatives.append(negative)
    return np.array(anchors), np.array(positives), np.array(negatives)

anchor, positive, negative = generate_ssl_triplets(sequences)

# 4. Define the encoder model
def create_encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(64)(x)
    x = Dense(32, activation='relu')(x)
    return Model(inputs, x)

encoder = create_encoder((window_size, 4))

# 5. Custom training loop with contrastive loss
def compute_loss(anchor_out, positive_out, negative_out, margin=1.0):
    pos_dist = tf.reduce_sum(tf.square(anchor_out - positive_out), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor_out - negative_out), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)

# 6. Training
epochs = 20
batch_size = 32
optimizer = Adam()

for epoch in range(epochs):
    indices = np.arange(len(anchor))
    np.random.shuffle(indices)

    total_loss = 0
    for i in range(0, len(anchor), batch_size):
        idx = indices[i:i+batch_size]
        a_batch = tf.convert_to_tensor(anchor[idx], dtype=tf.float32)
        p_batch = tf.convert_to_tensor(positive[idx], dtype=tf.float32)
        n_batch = tf.convert_to_tensor(negative[idx], dtype=tf.float32)

        with tf.GradientTape() as tape:
            a_out = encoder(a_batch, training=True)
            p_out = encoder(p_batch, training=True)
            n_out = encoder(n_batch, training=True)
            loss = compute_loss(a_out, p_out, n_out)

        grads = tape.gradient(loss, encoder.trainable_variables)
        optimizer.apply_gradients(zip(grads, encoder.trainable_variables))
        total_loss += loss.numpy()

    avg_loss = total_loss / (len(anchor) // batch_size)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
